In [1]:
import numpy as np
DTYPE = np.float64
# import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import os
import pickle
import gzip
from tqdm import tqdm
import sys
sys.path.insert(0, "../../packages")
import flory_
import math

In [2]:
# Flory Huggins Free Energy function
def floryHuggins(phi:DTYPE, chi:np.array):
    part_1 = np.sum(phi*np.log(phi))
    part_2 = 0

    for i in range(len(phi)):
        for j in range(i+1, len(phi)):
            part_2 += chi[i][j]*phi[i]*phi[j]

    return part_1 + part_2

In [38]:
# General Function for computing mergers of compartments
# Returns the concentrations of components in the merged and the unmerged compartments
# Stores them in phi_in_kmerged and phi_in_kunmerged respectively.
    

def mergers_4To2_combine3(concs:np.array, vols:np.array, chis, merged_compartments:list):
    # Find the unmerged compartment(s)
    expected = len(vols)*(len(vols)+1)//2
    actual = np.sum(merged_compartments)
    unmerged_compartment = expected - actual

    # print(concs)
    # print(vols)
    # print(unmerged_compartment)

    # Compute the merged volumes, stored in variable eta_merged
    # subtract -1 from the merged_compartments idxs to maintain python idxing
    eta_merged = 0
    for compartment in merged_compartments:
        eta_merged += vols[compartment-1]
    # print(eta_merged)

    # # Calculating the compositions of components in the merged compartment
    # # Taking a simple weighted average
    phi_1merged = (vols[merged_compartments[0]-1]*concs[0, merged_compartments[0]-1] + vols[merged_compartments[1]-1]*concs[0, merged_compartments[1]-1] + vols[merged_compartments[2]-1]*concs[0, merged_compartments[2]-1])/eta_merged
    phi_2merged = (vols[merged_compartments[0]-1]*concs[1, merged_compartments[0]-1] + vols[merged_compartments[1]-1]*concs[1, merged_compartments[1]-1] + vols[merged_compartments[2]-1]*concs[1, merged_compartments[2]-1])/eta_merged
    phi_3merged = (vols[merged_compartments[0]-1]*concs[2, merged_compartments[0]-1] + vols[merged_compartments[1]-1]*concs[2, merged_compartments[1]-1] + vols[merged_compartments[2]-1]*concs[2, merged_compartments[2]-1])/eta_merged
    phi_4merged = 1 - phi_1merged - phi_2merged - phi_3merged
    
    # print(phi_1merged+phi_2merged+phi_3merged+phi_4merged)    

    phi_in_kmerged = [phi_1merged, phi_2merged, phi_3merged, phi_4merged]
    phi_in_kunmerged = [concs[0, unmerged_compartment-1], concs[1, unmerged_compartment-1], concs[2, unmerged_compartment-1], concs[3, unmerged_compartment-1]]
    # print(phi_in_kmerged)
    # print(phi_in_kunmerged)

    # F_merged = eta_merged*floryHuggins(phi_in_kmerged, chis) + vols[unmerged_compartment-1]*floryHuggins(phi_in_kunmerged, chis)
    # # print(F_merged)

    return phi_in_kunmerged, phi_in_kmerged, eta_merged, unmerged_compartment

def mergers_4To3_combine3(concs:np.array, vols:np.array, chis, merged_compartments:list):
    eta_merged = 0
    for compartment in merged_compartments:
        eta_merged += vols[compartment-1]

    phi_1merged = (vols[merged_compartments[0]-1]*concs[0, merged_compartments[0]-1]) + (vols[merged_compartments[1]-1]*concs[0, merged_compartments[1]-1]) 
    phi_2merged = (vols[merged_compartments[0]-1]*concs[1, merged_compartments[0]-1]) + (vols[merged_compartments[1]-1]*concs[1, merged_compartments[1]-1]) 
    phi_3merged = (vols[merged_compartments[0]-1]*concs[2, merged_compartments[0]-1]) + (vols[merged_compartments[1]-1]*concs[2, merged_compartments[1]-1]) 
    phi_4merged = 1 -phi_1merged - phi_2merged - phi_3merged

    phi_in_kmerged = [phi_1merged, phi_2merged, phi_3merged, phi_4merged]
    # print(eta_merged)
    return phi_in_kmerged, eta_merged

def mergers_4To2_combine2And2(concs:np.array, vols:np.array, chis, merged_compartments_set1:list, merged_compartments_set2:list):
    # Set 1
    eta_merged_set1 = 0
    for compartment in merged_compartments_set1:
        eta_merged_set1 += vols[compartment-1]

    phi_1merged_set1 = (vols[merged_compartments_set1[0]-1]*concs[0, merged_compartments_set1[0]-1] + vols[merged_compartments_set1[1]-1]*concs[0, merged_compartments_set1[1]-1])/eta_merged_set1
    phi_2merged_set1 = (vols[merged_compartments_set1[0]-1]*concs[1, merged_compartments_set1[0]-1] + vols[merged_compartments_set1[1]-1]*concs[1, merged_compartments_set1[1]-1])/eta_merged_set1
    phi_3merged_set1 = (vols[merged_compartments_set1[0]-1]*concs[2, merged_compartments_set1[0]-1] + vols[merged_compartments_set1[1]-1]*concs[2, merged_compartments_set1[1]-1])/eta_merged_set1
    phi_4merged_set1 = 1 - phi_1merged_set1 - phi_2merged_set1 - phi_3merged_set1
    # phi_4merged_set1 = (vols[merged_compartments_set1[0]-1]*concs[3, merged_compartments_set1[0]-1] + vols[merged_compartments_set1[1]-1]*concs[3, merged_compartments_set1[1]-1])/eta_merged_set1
    phi_in_kset1 = [phi_1merged_set1, phi_2merged_set1, phi_3merged_set1, phi_4merged_set1]
    # Set2
    eta_merged_set2 = 0
    for compartment in merged_compartments_set2:
        eta_merged_set2 += vols[compartment-1]

    phi_1merged_set2 = (vols[merged_compartments_set2[0]-1]*concs[0, merged_compartments_set2[0]-1] + vols[merged_compartments_set2[1]-1]*concs[0, merged_compartments_set2[1]-1])/eta_merged_set2
    phi_2merged_set2 = (vols[merged_compartments_set2[0]-1]*concs[1, merged_compartments_set2[0]-1] + vols[merged_compartments_set2[1]-1]*concs[1, merged_compartments_set2[1]-1])/eta_merged_set2
    phi_3merged_set2 = (vols[merged_compartments_set2[0]-1]*concs[2, merged_compartments_set2[0]-1] + vols[merged_compartments_set2[1]-1]*concs[2, merged_compartments_set2[1]-1])/eta_merged_set2
    phi_4merged_set2 = 1 - phi_1merged_set2 - phi_2merged_set2 - phi_3merged_set2
    phi_in_kset2 = [phi_1merged_set2, phi_2merged_set2, phi_3merged_set2, phi_4merged_set2]

    # print(np.sum(phi_in_kset1), np.sum(phi_in_kset2), eta_merged_set1, eta_merged_set2)
    return phi_in_kset1, phi_in_kset2, eta_merged_set1, eta_merged_set2



In [49]:
def calculate_4To3_combine2(concs, vols, steps):
    merged_compartments_list = [np.array([1, 2], dtype=np.int64),
                                np.array([1, 3], dtype=np.int64),
                                np.array([1, 4], dtype=np.int64),
                                np.array([2, 3], dtype=np.int64),
                                np.array([2, 4], dtype=np.int64),
                                np.array([3, 4], dtype=np.int64),
                               ]
    
    for merged_compartments in merged_compartments_list:
        print(merged_compartments)
        
        phi_in_kmerged, eta_merged =  mergers_4To3_combine3(concs, vols, chis, merged_compartments)
        unmerged  = np.setdiff1d( np.array([1, 2, 3 ,4], dtype=np.int64), merged_compartments)
        
        f_initial = eta_merged*floryHuggins(phi_in_kmerged, chis) + vols[unmerged[0]-1]*floryHuggins(concs[unmerged[0]-1], chis) + vols[unmerged[1]-1]*floryHuggins(concs[unmerged[1]-1], chis)
    
        phi_global_ = np.array([phi_in_kmerged, concs[unmerged[0]-1], concs[unmerged[1]-1]])
        vols_ = np.array([eta_merged, vols[unmerged[0]-1], vols[unmerged[1]-1]])
    
        options = {
                        "num_part": 3,
                        "progress": False,
                        "max_steps": steps,  # disable progress bar, allow more steps
        }
        
        finder2 = flory_.CoexistingPhasesFinder(interaction, entropy, ensemble, **options)
        # finder2.reinitialize_from_omegas(-np.log(np.transpose(phi_global_)), vols_)
        finder2.reinitialize_from_phis(np.transpose(phi_global_), vols_)
        phases = finder2.run(progress=False).get_clusters()
        
        vols_m =  phases.volumes/np.sum(phases.volumes)
        fracs_m = phases.fractions
        
        f = []
        for idx, _ in enumerate(vols_m):
            f.append(vols_m[idx]*floryHuggins(fracs_m[idx], chis))
        f_best = np.sum(f)
        
        output_filepath = f"data/withFlory/phi_g{phi_global}/raw/X{X:.3f}/steps{steps}/mergers/4To3_combine2/{merged_compartments}/"
        output_filename = f"initial_and_best.pkl"
        if not os.path.exists(output_filepath):
            os.makedirs(output_filepath)
        output_file = os.path.join(output_filepath, output_filename)
        data_to_save = {
            "merged_compartments": merged_compartments,
            "initial_guess": [phi_global_, vols_],
            "F_initial": f_initial,
            "best_location": [fracs_m[0], vols_m[0]],
            "F_best": f_best,
            "metadata": {
                "initial_guess": "stores the initial merged guesses for concs and volumes",
                "F_initial": "stores the initial free energy for the guess",
                "best_location": "stores flory's output for the local minima positions [phi_flory, eta_flory] -> takes the first row of the phi matrix and the first entry of the volume array",
                "F_best": "stores the best free energy"
            }
        }
    
        # print(data_to_save)
        with gzip.open(output_file, "wb") as file:
            pickle.dump(data_to_save, file, protocol=pickle.HIGHEST_PROTOCOL)
        print(f_initial, f_best)
    # print()
    # print()

    
def calculate_4To2_combine3(concs, vols, steps):
    merged_compartments_list = [np.array([1, 2, 3], dtype=np.int64), 
                                np.array([1, 2, 4], dtype=np.int64),
                                np.array([1, 3, 4], dtype=np.int64),
                                np.array([2, 3, 4], dtype=np.int64)]
    
    for merged_compartments in merged_compartments_list:
        print(merged_compartments)
        phi_in_kunmerged, phi_in_kmerged, eta_merged, unmerged_compartment = mergers_4To2_combine3(concs, vols, chis, merged_compartments)
        f_initial = eta_merged*floryHuggins(phi_in_kmerged, chis) + (1-eta_merged)*floryHuggins(phi_in_kunmerged, chis)
        
        phi_global_ = np.array([phi_in_kmerged, phi_in_kunmerged])
        vols_ = np.array([eta_merged, 1-eta_merged])
        
        options = {
                        "num_part": 2,
                        "progress": False,
                        "max_steps": steps,  # disable progress bar, allow more steps
        }
        
        finder2 = flory_.CoexistingPhasesFinder(interaction, entropy, ensemble, **options)
        # finder2.reinitialize_from_omegas(-np.log(np.transpose(phi_global_)), vols_)
        finder2.reinitialize_from_phis(np.transpose(phi_global_), vols_)
        phases = finder2.run(progress=False).get_clusters()
    
        vols_m =  phases.volumes/np.sum(phases.volumes)
        fracs_m = phases.fractions
    
        f = []
        for idx, _ in enumerate(vols_m):
            f.append(vols_m[idx]*floryHuggins(fracs_m[idx], chis))
        f_best = np.sum(f)
    
        output_filepath = f"data/withFlory/phi_g{phi_global}/raw/X{X:.3f}/steps{steps}/mergers/4To2_combine3/{merged_compartments}/"
        output_filename = f"initial_and_best.pkl"
        if not os.path.exists(output_filepath):
            os.makedirs(output_filepath)
        output_file = os.path.join(output_filepath, output_filename)
        data_to_save = {
            "merged_compartments": merged_compartments,
            "initial_guess": [phi_in_kmerged, eta_merged],
            "F_initial": f_initial,
            "best_location": [fracs_m[0], vols_m[0]],
            "F_best": f_best,
            "metadata": {
                "initial_guess": "stores the initial merged guesses [phi_merged, eta_merged]",
                "F_initial": "stores the initial free energy for the guess",
                "best_location": "stores flory's output for the local minima positions [phi_flory, eta_flory] -> takes the first row of the phi matrix and the first entry of the volume array",
                "F_best": "stores the best free energy"
            }
        }
    
        # print(data_to_save)
        with gzip.open(output_file, "wb") as file:
            pickle.dump(data_to_save, file, protocol=pickle.HIGHEST_PROTOCOL)
        print(f_initial, f_best)
    # print()
    # print()


def calculate_4To2_combine2And2(concs, vols, steps):
    merged_compartments_set1 = [np.array([1, 2], dtype=np.int64), 
                                np.array([1, 3], dtype=np.int64),
                                np.array([1, 4], dtype=np.int64)]
    
    merged_compartments_set2 = [np.array([3, 4], dtype=np.int64), 
                                np.array([2, 4], dtype=np.int64),
                                np.array([2, 3], dtype=np.int64)]
    
    for merged_compartments in (zip(merged_compartments_set1, merged_compartments_set2)):
        print(merged_compartments[0], merged_compartments[1])
        phi_in_kset1, phi_in_kset2, eta_merged_set1, eta_merged_set2 = mergers_4To2_combine2And2(concs, vols, chis, merged_compartments[0], merged_compartments[1])
        f_initial = eta_merged_set1*floryHuggins(phi_in_kset1, chis) + eta_merged_set2*floryHuggins(phi_in_kset2, chis)
        # print(eta_merged_set1+eta_merged_set2)
    
        phi_global_ = np.array([phi_in_kset1, phi_in_kset2])
        vols_ = np.array([eta_merged_set1, eta_merged_set2])
    
        options = {
                        "num_part": 2,
                        "progress": False,
                        "max_steps": steps,  # disable progress bar, allow more steps
        }
        
        finder2 = flory_.CoexistingPhasesFinder(interaction, entropy, ensemble, **options)
        # finder2.reinitialize_from_omegas(-np.log(np.transpose(phi_global_)), vols_)
        finder2.reinitialize_from_phis(np.transpose(phi_global_), vols_)
        phases = finder2.run(progress=False).get_clusters()
    
        vols_m =  phases.volumes/np.sum(phases.volumes)
        fracs_m = phases.fractions
    
        f = []
        for idx, _ in enumerate(vols_m):
            f.append(vols_m[idx]*floryHuggins(fracs_m[idx], chis))
        f_best = np.sum(f)
    
        output_filepath = f"data/withFlory/phi_g{phi_global}/raw/X{X:.3f}/steps{steps}/mergers/4To2_combine2And2/{merged_compartments[0]} {merged_compartments[1]}/"
        output_filename = f"initial_and_best.pkl"
        if not os.path.exists(output_filepath):
            os.makedirs(output_filepath)
        output_file = os.path.join(output_filepath, output_filename)
        data_to_save = {
            "merged_compartments": merged_compartments,
            "initial_guess_set1": [phi_in_kset1, eta_merged_set1],
            "initial_guess_set2": [phi_in_kset2, eta_merged_set2],
            "F_initial": f_initial,
            "best_location": [fracs_m[0], vols_m[0]],
            "F_best": f_best,
            "metadata": {
                "initial_guess_set1": "stores the initial merged guesses [phi_set1, eta_set1]-> weighted averages after merging the first set of 2 compartments",
                "initial_guess_set2": "stores the initial merged guesses [phi_set2, eta_set2]-> weighted averages after merging the second set of 2 compartments",
                "F_initial": "stores the initial free energy for the guess",
                "best_location": "stores flory's output for the local minima positions [phi_flory, eta_flory] -> takes the first row of the phi matrix and the first entry of the volume array",
                "F_best": "stores the best free energy"
            }
        }
        
        # print(data_to_save)
        with gzip.open(output_file, "wb") as file:
            pickle.dump(data_to_save, file, protocol=pickle.HIGHEST_PROTOCOL)
        print(f_initial, f_best)

In [50]:
phi_global = np.array([0.3, 0.2, 0.45, 0.05], dtype=DTYPE)
Xs = np.arange(1, 10.1, 0.1)

N_STEPS = [100000]

In [51]:
for X in (Xs):
    for steps in N_STEPS:
        print()
        print(X)
        
        chis = np.array([[0, 3, 3, 3+X],
                         [3, 0, 3+X, 3],
                         [3, 3+X, 0, 3],
                         [3+X, 3, 3, 0]], dtype = DTYPE)
    
        n_components = 4
        free_energy = flory_.free_energy.FloryHuggins(n_components, chis)
        interaction = free_energy.interaction
        entropy = free_energy.entropy
        ensemble = flory_.CanonicalEnsemble(n_components, phi_global)
        options = {"num_part": 32, "progress": False, "max_steps": 1000000}
        
        finder = flory_.CoexistingPhasesFinder(interaction, entropy, ensemble, **options)
        phases = finder.run(progress=False)
        
        # Get the number of phases
        vols = phases.get_clusters().volumes
        concs = phases.get_clusters().fractions
        
        # Normalize volumes
        vols = vols/np.sum(vols)
        if concs.shape[0] == 4: # force 4 phase, else skip    
            calculate_4To3_combine2(concs, vols, steps)
            calculate_4To2_combine3(concs, vols, steps)
            calculate_4To2_combine2And2(concs, vols, steps)
           
        else:
            print(f"Skipped for {chis}")


1.0
Skipped for [[0. 3. 3. 4.]
 [3. 0. 4. 3.]
 [3. 4. 0. 3.]
 [4. 3. 3. 0.]]

1.1
Skipped for [[0.  3.  3.  4.1]
 [3.  0.  4.1 3. ]
 [3.  4.1 0.  3. ]
 [4.1 3.  3.  0. ]]

1.2000000000000002
Skipped for [[0.  3.  3.  4.2]
 [3.  0.  4.2 3. ]
 [3.  4.2 0.  3. ]
 [4.2 3.  3.  0. ]]

1.3000000000000003
Skipped for [[0.  3.  3.  4.3]
 [3.  0.  4.3 3. ]
 [3.  4.3 0.  3. ]
 [4.3 3.  3.  0. ]]

1.4000000000000004
Skipped for [[0.  3.  3.  4.4]
 [3.  0.  4.4 3. ]
 [3.  4.4 0.  3. ]
 [4.4 3.  3.  0. ]]

1.5000000000000004
Skipped for [[0.  3.  3.  4.5]
 [3.  0.  4.5 3. ]
 [3.  4.5 0.  3. ]
 [4.5 3.  3.  0. ]]

1.6000000000000005
Skipped for [[0.  3.  3.  4.6]
 [3.  0.  4.6 3. ]
 [3.  4.6 0.  3. ]
 [4.6 3.  3.  0. ]]

1.7000000000000006
Skipped for [[0.  3.  3.  4.7]
 [3.  0.  4.7 3. ]
 [3.  4.7 0.  3. ]
 [4.7 3.  3.  0. ]]

1.8000000000000007
Skipped for [[0.  3.  3.  4.8]
 [3.  0.  4.8 3. ]
 [3.  4.8 0.  3. ]
 [4.8 3.  3.  0. ]]

1.9000000000000008
Skipped for [[0.  3.  3.  4.9]
 [3.  0.  4.9 